# 패키지 설치

In [69]:
!pip install pandas fiona shapely pyproj rtree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import geopandas as gpd
import pandas as pd

# 집계구 데이터 불러오기

In [104]:
standard_data = gpd.read_file('/content/drive/MyDrive/청년인재팀플/data/QGIS/집계구/서울집계구_5179.shp')

In [105]:
standard_data.crs = "epsg:5179"
standard_data = standard_data.to_crs(epsg=5179)

In [106]:
standard_data.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'geometry'], dtype='object')

In [107]:
standard_data= standard_data.astype({'TOT_REG_CD':'int64'})
standard_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19153 entries, 0 to 19152
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   TOT_REG_CD  19153 non-null  int64   
 1   ADM_NM      19153 non-null  object  
 2   ADM_CD      19153 non-null  object  
 3   geometry    19153 non-null  geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 598.7+ KB


# QGIS 데이터 불러오기(학교, 버스)

In [76]:
life_data = gpd.read_file('/content/drive/MyDrive/청년인재팀플/data/QGIS/사고지점_독립변수 속성테이블 결합/사고X랜덤지점/2000개/사고X2000+hos+bus+sub+univ+초중고+CarLane+police+횡+방지시설+어린이+노인.shp')

In [77]:
life_data['geometry'] = life_data['geometry'].centroid

In [78]:
life_data = life_data.drop(['STAT_CDE', 'MGRNU', 'A058_KND_C', 'EVE_CDE', 'OD_PE_CDE', 'GU_CDE',
       'NW_PE_CDE', 'WORK_CDE', 'VIEW_CDE', 'ROD_GBN_CD', 'TFC_BSS_CD',
       'SIXID', 'ESB_YMD', 'CAE_YMD', 'HISID', 'CTK_MGRNU', 'LINE_MGRNU',
       'A058_KND2_', 'FRM_CDE', 'LENX', 'RN_CDE', 'MNG_AGEN', 'distance',
       'angle', 'xcoord', 'ycoord'],axis=1)

In [79]:
qgis_stand = standard_data.sjoin(life_data, how="inner", predicate='contains')

In [80]:
qgis_stand.shape

(1995, 16)

In [102]:
qgis_stand.isna().sum()

TOT_REG_CD     0
ADM_NM         0
ADM_CD         0
geometry       0
index_right    0
hospital       0
bus            0
subway         0
univ           0
초중고            0
CarLane100     0
police         0
횡단보도           0
무단횡단방          0
어린이보호          0
노인보호구          0
dtype: int64

# 생활인구 데이터 결합 (야간)

In [113]:
pd.merge(people_data,standard_data,how='left',left_on='TOT_REG_CD',right_on='TOT_REG_CD').isna().sum()

TOT_REG_CD       0
ADM_NM_x         0
ADM_CD_x         0
xcoord           0
ycoord           0
집계구코드            0
주간야간             0
행정동코드            0
총생활인구수           0
남자 10대 인구수       0
남자 20대 인구수       0
남자 30대 인구수       0
남자 40대 인구수       0
남자 50대 인구수       0
남자 60대 인구수       0
남자 70대 이상 인구수    0
여자 10대 인구수       0
여자 20대 인구수       0
여자 30대 인구수       0
여자 40대 인구수       0
여자 50대 인구수       0
여자 60대 인구수       0
여자 70대 이상 인구수    0
기준일ID            0
월1               0
ADM_NM_y         0
ADM_CD_y         0
geometry         0
dtype: int64

In [81]:
people_data=pd.read_csv('/content/drive/MyDrive/청년인재팀플/data/QGIS_생활인구 야간 평균.csv',encoding='cp949')

In [82]:
people_data.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'xcoord', 'ycoord', '집계구코드', '주간야간',
       '행정동코드', '총생활인구수', '남자 10대 인구수', '남자 20대 인구수', '남자 30대 인구수',
       '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수', '남자 70대 이상 인구수', '여자 10대 인구수',
       '여자 20대 인구수', '여자 30대 인구수', '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수',
       '여자 70대 이상 인구수', '기준일ID', '월1'],
      dtype='object')

In [83]:
total = pd.merge(qgis_stand,people_data,how='left',left_on='TOT_REG_CD',right_on='TOT_REG_CD')

In [84]:
total.shape

(1995, 40)

In [110]:
total.isna().sum()

TOT_REG_CD          0
ADM_NM_x            0
ADM_CD_x            0
geometry            0
index_right         0
hospital            0
bus                 0
subway              0
univ                0
초중고                 0
CarLane100          0
police              0
횡단보도                0
무단횡단방               0
어린이보호               0
노인보호구               0
ADM_NM_y         1878
ADM_CD_y         1878
xcoord           1878
ycoord           1878
집계구코드            1878
주간야간             1878
행정동코드            1878
총생활인구수           1878
남자 10대 인구수       1878
남자 20대 인구수       1878
남자 30대 인구수       1878
남자 40대 인구수       1878
남자 50대 인구수       1878
남자 60대 인구수       1878
남자 70대 이상 인구수    1878
여자 10대 인구수       1878
여자 20대 인구수       1878
여자 30대 인구수       1878
여자 40대 인구수       1878
여자 50대 인구수       1878
여자 60대 인구수       1878
여자 70대 이상 인구수    1878
기준일ID            1878
월1               1878
dtype: int64

In [85]:
night_data = total.drop(['ADM_NM_x', 'ADM_CD_x', 'index_right','ADM_NM_y', 'ADM_CD_y',
      '집계구코드', '주간야간', '행정동코드','기준일ID', '월1','xcoord',
       'ycoord'],axis=1)

In [86]:
night_data['야간 남자 20-50대'] = night_data['남자 20대 인구수']+night_data['남자 30대 인구수']+night_data['남자 40대 인구수']+night_data['남자 50대 인구수']

In [87]:
night_data['야간 여자 20-50대'] = night_data['여자 20대 인구수']+night_data['여자 30대 인구수']+night_data['여자 40대 인구수']+night_data['여자 50대 인구수']

In [88]:
night_data['야간 남자 60대 이상'] = night_data['남자 60대 인구수']+night_data['남자 70대 이상 인구수']
night_data['야간 여자 60대 이상'] = night_data['여자 60대 인구수']+night_data['여자 70대 이상 인구수']

In [89]:
night_data.rename(columns={'총생활인구수':'야간총생활인구수','남자 10대 인구수':'야간 남자 10대 이하','여자 10대 인구수':'야간 여자 10대 이하'},inplace=True)

In [90]:
night_data = night_data.drop(['남자 20대 인구수', '남자 30대 인구수', '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수',
       '남자 70대 이상 인구수','여자 20대 인구수', '여자 30대 인구수',
       '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수', '여자 70대 이상 인구수'],axis=1)

In [91]:
night_data.shape

(1995, 20)

# 생활인구 데이터 (주간)

In [92]:
people_data2=pd.read_csv('/content/drive/MyDrive/청년인재팀플/data/QGIS_생활인구 주간 평균.csv',encoding='cp949')

In [93]:
people_data2['주간 남자 20-50대'] = people_data2['남자 20대 인구수']+people_data2['남자 30대 인구수']+people_data2['남자 40대 인구수']+people_data2['남자 50대 인구수']
people_data2['주간 여자 20-50대'] = people_data2['여자 20대 인구수']+people_data2['여자 30대 인구수']+people_data2['여자 40대 인구수']+people_data2['여자 50대 인구수']
people_data2['주간 남자 60대 이상'] = people_data2['남자 60대 인구수']+people_data2['남자 70대 이상 인구수']
people_data2['주간 여자 60대 이상'] = people_data2['여자 60대 인구수']+people_data2['여자 70대 이상 인구수']
people_data2.rename(columns={'총생활인구수':'주간총생활인구수','남자 10대 인구수':'주간 남자 10대 이하','여자 10대 인구수':'주간 여자 10대 이하'},inplace=True)
people_data2 = people_data2.drop(['남자 20대 인구수', '남자 30대 인구수', '남자 40대 인구수', '남자 50대 인구수', '남자 60대 인구수',
       '남자 70대 이상 인구수','여자 20대 인구수', '여자 30대 인구수',
       '여자 40대 인구수', '여자 50대 인구수', '여자 60대 인구수', '여자 70대 이상 인구수'],axis=1)

In [94]:
people_data2.columns

Index(['TOT_REG_CD', 'ADM_NM', 'ADM_CD', 'xcoord', 'ycoord', '집계구코드', '주간야간',
       '행정동코드', '주간총생활인구수', '주간 남자 10대 이하', '주간 여자 10대 이하', '기준일ID', '월1',
       '주간 남자 20-50대', '주간 여자 20-50대', '주간 남자 60대 이상', '주간 여자 60대 이상'],
      dtype='object')

In [95]:
people_data2.shape

(5260, 17)

In [96]:
final_data = pd.merge(night_data,people_data2,how='left',left_on='TOT_REG_CD',right_on='TOT_REG_CD')

In [97]:
final_data = final_data.drop(['ADM_NM', 'ADM_CD', 'xcoord', 'ycoord','기준일ID', '월1'],axis=1)

In [98]:
final_data.columns

Index(['TOT_REG_CD', 'geometry', 'hospital', 'bus', 'subway', 'univ', '초중고',
       'CarLane100', 'police', '횡단보도', '무단횡단방', '어린이보호', '노인보호구', '야간총생활인구수',
       '야간 남자 10대 이하', '야간 여자 10대 이하', '야간 남자 20-50대', '야간 여자 20-50대',
       '야간 남자 60대 이상', '야간 여자 60대 이상', '집계구코드', '주간야간', '행정동코드', '주간총생활인구수',
       '주간 남자 10대 이하', '주간 여자 10대 이하', '주간 남자 20-50대', '주간 여자 20-50대',
       '주간 남자 60대 이상', '주간 여자 60대 이상'],
      dtype='object')

In [99]:
final_data.shape

(1995, 30)

In [100]:
final_data.isna().sum()

TOT_REG_CD         0
geometry           0
hospital           0
bus                0
subway             0
univ               0
초중고                0
CarLane100         0
police             0
횡단보도               0
무단횡단방              0
어린이보호              0
노인보호구              0
야간총생활인구수        1878
야간 남자 10대 이하    1878
야간 여자 10대 이하    1878
야간 남자 20-50대    1878
야간 여자 20-50대    1878
야간 남자 60대 이상    1878
야간 여자 60대 이상    1878
집계구코드           1878
주간야간            1878
행정동코드           1878
주간총생활인구수        1878
주간 남자 10대 이하    1878
주간 여자 10대 이하    1878
주간 남자 20-50대    1878
주간 여자 20-50대    1878
주간 남자 60대 이상    1878
주간 여자 60대 이상    1878
dtype: int64

In [101]:
#final_data.to_csv('/content/drive/MyDrive/청년인재팀플/data/사고다발지아닌_생활인구_최종데이터.csv')